In [84]:
import pandas as pd
import boto3

from dotenv import load_dotenv
import os
from io import StringIO
pd.set_option("display.max_columns", 50)
import seaborn as sns
import matplotlib.pyplot as plt

In [91]:
aws_access_key_id = os.getenv("AWS_ACCESS_KEY")
aws_secret_key = os.getenv("AWS_SECRET_KEY")

In [95]:
def read_csv_from_s3(bucket: str, path: str, filename: str) -> pd.DataFrame:
    """
    Reads a CSV file from an S3 bucket and returns it as a pandas DataFrame.

    Parameters:
        bucket (str): 
            The name of the S3 bucket.
        path (str): 
            The path within the S3 bucket where the file is located.
        filename (str): 
            The name of the CSV file.

    Returns:
        pd.DataFrame: DataFrame containing the contents of the CSV file.

    """
    s3 = boto3.client('s3', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_key)
    
    full_path = f'{path}{filename}'
    
    object = s3.get_object(Bucket=bucket, Key=full_path)
    object = object['Body'].read().decode('utf-8')
    output_df = pd.read_csv(StringIO(object))
    
    return output_df

In [117]:
s3 = boto3.client('s3', aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_key)
bucket = 'cubix-chicago-taxi-rr'

community_areas_path = 'transformed_data/community_areas/'
company_path = 'transformed_data/company/'
date_path = 'transformed_data/date/'
payment_type_path = 'transformed_data/payment_type/'
taxi_trips_path = 'transformed_data/taxi_trips/'
weather_path = 'transformed_data/weather/'

In [118]:
community_areas = read_csv_from_s3(bucket=bucket, path=community_areas_path, filename='community_areas_master.csv')
company = read_csv_from_s3(bucket=bucket, path=company_path, filename='company_master.csv')
date = read_csv_from_s3(bucket=bucket, path=date_path, filename='data_dimension.csv')
payment_type = read_csv_from_s3(bucket=bucket, path=payment_type_path, filename='payment_type_master.csv')

In [127]:
trips_list = []
weather_list = []

In [129]:
trips_list = []

for file in s3.list_objects(Bucket=bucket, Prefix=taxi_trips_path)['Contents']:
    taxi_trip_key = file ['Key']

    if taxi_trip_key.split("/")[-1].strip() !='':
        if taxi_trip_key.split('.')[1] == 'csv':
            
            filename = taxi_trip_key.split("/")[-1]
            #print(filename)
            trip = read_csv_from_s3(bucket, taxi_trips_path, filename)

            trips_list.append(trip)
            print(f"{filename} has been added.")

taxi_2023-12-03.csv has been added.
taxi_2023-12-04.csv has been added.
taxi_2023-12-05.csv has been added.
taxi_2023-12-06.csv has been added.
taxi_2023-12-07.csv has been added.
taxi_2023-12-08.csv has been added.
taxi_2023-12-09.csv has been added.
taxi_2023-12-10.csv has been added.
taxi_2023-12-11.csv has been added.
taxi_2023-12-12.csv has been added.
taxi_2023-12-13.csv has been added.
taxi_2023-12-14.csv has been added.


In [130]:
trips = pd.concat(trips_list, ignore_index=True)

In [155]:
trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,pickup_community_area_id,dropoff_community_area_id,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
0,ff109e00fb81239606eca1acec411db0eb3c70df,31261f6e7fc645eff98c7964c7ea71a0ea7e387a6bd7f8...,2023-12-03T23:45:00.000,2023-12-03T23:45:00.000,360,0.50,5.00,0.00,0.0,4.5,9.50,8,32,41.892508,-87.626215,41.884987,-87.620993,2023-12-03 23:00:00,1,2
1,f60813d29947103bf21fe771ff8ae30e3b59d062,8063b041e0d6272293d52b8d81343464e62e7fa3a9e900...,2023-12-03T23:45:00.000,2023-12-04T00:00:00.000,1312,17.61,43.25,9.75,0.0,5.0,58.50,76,8,41.980264,-87.913625,41.899602,-87.633308,2023-12-03 23:00:00,1,7
2,e3fded002873476bdc0b39bde244c406506d46c8,093e9e4c05ea53bf75c51763839d5f5bd5d1785c11ee5e...,2023-12-03T23:45:00.000,2023-12-03T23:45:00.000,268,0.91,5.75,4.00,0.0,1.5,11.75,8,32,41.892508,-87.626215,41.880994,-87.632746,2023-12-03 23:00:00,1,8
3,e35a449ec6ea1fa64a22c0db18a20d2bb25692e4,2e82e26afb77e809fe4a44b02a152bdc079623600ae1b7...,2023-12-03T23:45:00.000,2023-12-04T00:00:00.000,1080,0.60,26.75,5.00,0.0,6.0,37.75,56,33,41.792592,-87.769615,41.857184,-87.620335,2023-12-03 23:00:00,1,2
4,e36a934e656a0c8ea395b331c17a903906402642,02ef8f01232b1b1828f4e5e1b8e8a85cd71b67c449afaf...,2023-12-03T23:45:00.000,2023-12-04T00:15:00.000,1692,10.07,29.00,0.00,0.0,0.0,29.00,44,34,41.740206,-87.615970,41.842076,-87.633973,2023-12-03 23:00:00,5,8


In [156]:
weather_list = []

for file in s3.list_objects(Bucket=bucket, Prefix=weather_path)['Contents']:
    weather_key = file['Key']
    
    if weather_key.split("/")[-1].strip() !='':
        if weather_key.split('.')[1] == 'csv':
            
            filename = weather_key.split('/')[-1]
            #print(filename)
            weather_daily = read_csv_from_s3(bucket, weather_path, filename)

            weather_list.append(weather_daily)
            print(f"{filename} has been added.")

weather_2023-12-03.csv has been added.
weather_2023-12-04.csv has been added.
weather_2023-12-05.csv has been added.
weather_2023-12-06.csv has been added.
weather_2023-12-07.csv has been added.
weather_2023-12-08.csv has been added.
weather_2023-12-09.csv has been added.
weather_2023-12-10.csv has been added.
weather_2023-12-11.csv has been added.
weather_2023-12-12.csv has been added.
weather_2023-12-13.csv has been added.
weather_2023-12-14.csv has been added.


In [157]:
weather = pd.concat(weather_list, ignore_index=True)

In [158]:
weather.head()

,datetime,temperature,wind_speed,precipitation,rain
0,2023-12-03 00:00:00,4.8,7.9,0.0,0.0
1,2023-12-03 01:00:00,4.6,6.9,0.0,0.0
2,2023-12-03 02:00:00,4.5,8.3,0.0,0.0
3,2023-12-03 03:00:00,4.7,8.5,0.0,0.0
4,2023-12-03 04:00:00,4.7,9.2,0.0,0.0


In [159]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime       288 non-null    object 
 1   temperature    288 non-null    float64
 2   wind_speed     288 non-null    float64
 3   precipitation  288 non-null    float64
 4   rain           288 non-null    float64
dtypes: float64(4), object(1)
memory usage: 11.4+ KB


# Join them together

In [160]:
print(weather.columns)
print(weather.head())


Index(['datetime', 'temperature', 'wind_speed', 'precipitation', 'rain'], dtype='object')
              datetime  temperature  wind_speed  precipitation  rain
0  2023-12-03 00:00:00          4.8         7.9            0.0   0.0
1  2023-12-03 01:00:00          4.6         6.9            0.0   0.0
2  2023-12-03 02:00:00          4.5         8.3            0.0   0.0
3  2023-12-03 03:00:00          4.7         8.5            0.0   0.0
4  2023-12-03 04:00:00          4.7         9.2            0.0   0.0


In [161]:
if 'datetime' in weather.columns:
    print('Datetime column exists.')
else:
    print('Datetime column is missing.')

Datetime column exists.


In [237]:
trips_full = pd.merge(trips, weather, left_on='datetime_for_weather', right_on='datetime', how='inner')
trips_full = trips_full.drop(columns=['datetime'])

In [238]:
trips_full = pd.merge(trips_full, company, left_on='company_id', right_on='company_id', how='inner')
trips_full = trips_full.drop(columns=['company_id'])

In [239]:
trips_full = pd.merge(trips_full, payment_type, left_on='payment_type_id', right_on='payment_type_id', how='inner')
trips_full = trips_full.drop(columns=['payment_type_id'])

In [240]:
trips_full = pd.merge(trips_full, community_areas, left_on='pickup_community_area_id', right_on='area_code', how='inner')
trips_full = trips_full.drop(columns=['pickup_community_area_id', 'area_code'])
trips_full.rename(columns={'community_name':'pickup_community_area_name'}, inplace=True)

In [241]:
trips_full = pd.merge(trips_full, community_areas, left_on='dropoff_community_area_id', right_on='area_code', how='inner')
trips_full = trips_full.drop(columns=['dropoff_community_area_id','area_code'])
trips_full.rename(columns={'community_name':'dropoff_community_area_name'}, inplace=True)

In [242]:
date['date'] = pd.to_datetime(date['date'])
trips_full['trip_end_timestamp'] = pd.to_datetime(trips_full['trip_end_timestamp'])

trips_full['trip_start_date'] = trips_full['trip_end_timestamp'].dt.date   # DATE(column)
trips_full['trip_start_date'] = pd.to_datetime(trips_full['trip_start_date'])


In [243]:
trips_full.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191967 entries, 0 to 191966
Data columns (total 25 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   trip_id                      191967 non-null  object        
 1   taxi_id                      191967 non-null  object        
 2   trip_start_timestamp         191967 non-null  object        
 3   trip_end_timestamp           191967 non-null  datetime64[ns]
 4   trip_seconds                 191967 non-null  int64         
 5   trip_miles                   191967 non-null  float64       
 6   fare                         191967 non-null  float64       
 7   tips                         191967 non-null  float64       
 8   tolls                        191967 non-null  float64       
 9   extras                       191967 non-null  float64       
 10  trip_total                   191967 non-null  float64       
 11  pickup_centroid_latitude  

In [244]:
trips_full.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,temperature,wind_speed,precipitation,rain,company,payment_type,pickup_community_area_name,dropoff_community_area_name,trip_start_date
0,ff109e00fb81239606eca1acec411db0eb3c70df,31261f6e7fc645eff98c7964c7ea71a0ea7e387a6bd7f8...,2023-12-03T23:45:00.000,2023-12-03 23:45:00,360,0.50,5.00,0.00,0.0,4.5,9.50,41.892508,-87.626215,41.884987,-87.620993,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,Taxi Affiliation Services,Credit Card,Near North Side,Loop,2023-12-03
1,f60813d29947103bf21fe771ff8ae30e3b59d062,8063b041e0d6272293d52b8d81343464e62e7fa3a9e900...,2023-12-03T23:45:00.000,2023-12-04 00:00:00,1312,17.61,43.25,9.75,0.0,5.0,58.50,41.980264,-87.913625,41.899602,-87.633308,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,Taxicab Insurance Agency Llc,Credit Card,O'Hare,Near North Side,2023-12-04
2,e3fded002873476bdc0b39bde244c406506d46c8,093e9e4c05ea53bf75c51763839d5f5bd5d1785c11ee5e...,2023-12-03T23:45:00.000,2023-12-03 23:45:00,268,0.91,5.75,4.00,0.0,1.5,11.75,41.892508,-87.626215,41.880994,-87.632746,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,City Service,Credit Card,Near North Side,Loop,2023-12-03
3,e35a449ec6ea1fa64a22c0db18a20d2bb25692e4,2e82e26afb77e809fe4a44b02a152bdc079623600ae1b7...,2023-12-03T23:45:00.000,2023-12-04 00:00:00,1080,0.60,26.75,5.00,0.0,6.0,37.75,41.792592,-87.769615,41.857184,-87.620335,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,Taxi Affiliation Services,Credit Card,Garfield Ridge,Near South Side,2023-12-04
4,e36a934e656a0c8ea395b331c17a903906402642,02ef8f01232b1b1828f4e5e1b8e8a85cd71b67c449afaf...,2023-12-03T23:45:00.000,2023-12-04 00:15:00,1692,10.07,29.00,0.00,0.0,0.0,29.00,41.740206,-87.615970,41.842076,-87.633973,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,City Service,Prcard,Chatham,Armour Square,2023-12-04


In [245]:
trips_full = pd.merge(trips_full, date, left_on='trip_start_date', right_on='date', how='inner')
trips_full = trips_full.drop(columns=['date'])

In [246]:
trips_full.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,temperature,wind_speed,precipitation,rain,company,payment_type,pickup_community_area_name,dropoff_community_area_name,trip_start_date,year,month,day,day_of_week,is_weekend
0,ff109e00fb81239606eca1acec411db0eb3c70df,31261f6e7fc645eff98c7964c7ea71a0ea7e387a6bd7f8...,2023-12-03T23:45:00.000,2023-12-03 23:45:00,360,0.50,5.00,0.00,0.0,4.5,9.50,41.892508,-87.626215,41.884987,-87.620993,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,Taxi Affiliation Services,Credit Card,Near North Side,Loop,2023-12-03,2023,12,3,7,True
1,f60813d29947103bf21fe771ff8ae30e3b59d062,8063b041e0d6272293d52b8d81343464e62e7fa3a9e900...,2023-12-03T23:45:00.000,2023-12-04 00:00:00,1312,17.61,43.25,9.75,0.0,5.0,58.50,41.980264,-87.913625,41.899602,-87.633308,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,Taxicab Insurance Agency Llc,Credit Card,O'Hare,Near North Side,2023-12-04,2023,12,4,1,False
2,e3fded002873476bdc0b39bde244c406506d46c8,093e9e4c05ea53bf75c51763839d5f5bd5d1785c11ee5e...,2023-12-03T23:45:00.000,2023-12-03 23:45:00,268,0.91,5.75,4.00,0.0,1.5,11.75,41.892508,-87.626215,41.880994,-87.632746,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,City Service,Credit Card,Near North Side,Loop,2023-12-03,2023,12,3,7,True
3,e35a449ec6ea1fa64a22c0db18a20d2bb25692e4,2e82e26afb77e809fe4a44b02a152bdc079623600ae1b7...,2023-12-03T23:45:00.000,2023-12-04 00:00:00,1080,0.60,26.75,5.00,0.0,6.0,37.75,41.792592,-87.769615,41.857184,-87.620335,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,Taxi Affiliation Services,Credit Card,Garfield Ridge,Near South Side,2023-12-04,2023,12,4,1,False
4,e36a934e656a0c8ea395b331c17a903906402642,02ef8f01232b1b1828f4e5e1b8e8a85cd71b67c449afaf...,2023-12-03T23:45:00.000,2023-12-04 00:15:00,1692,10.07,29.00,0.00,0.0,0.0,29.00,41.740206,-87.615970,41.842076,-87.633973,2023-12-03 23:00:00,3.6,19.1,0.0,0.0,City Service,Prcard,Chatham,Armour Square,2023-12-04,2023,12,4,1,False
